In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5
import numpy as np
import time
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import analysis.switching as sw

In [3]:
from experiments.Pulse_Switching_CSHE_SWR import *

In [4]:
exp = SWRExperiment()
exp.sample = "CSHE5 - C2R3"
exp.comment = "Switching Probability - P to AP - 5ns"
exp.polarity = -1 # 1: AP to P; -1: P to AP
exp.field.value = 0.0081
exp.attempts = 1 << 14
exp.pulse_duration.value = 5e-9 # Fixed
exp.reset_amplitude = 0.7
exp.reset_duration = 5e-9
exp.settle_delay = 200e-6
exp.init_instruments()

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die5-C2R3\CSHE5-C2R3-P2AP_2016-07-27_SWR_5ns.h5")
edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)

In [6]:
V0 = 0.5
voltages_list = V0*np.arange(1.0,0.2,-0.1)
max_points = 1<<18
threshold = 0.36
start_state = 0 if exp.polarity<0 else 1

In [7]:
finish = False
for volt in tqdm_notebook(voltages_list, total=len(voltages_list)):
    if finish:
        print("Reached maximum points. Exit.")
        break
    print("=========================")
    print("Now at: {}.".format(volt))
    t1 = time.time()
    exp.pulse_voltage.value = volt
    forward = False
    count = 0
    while not forward:
        count = count + 1
        print("Measurement count = %d" %count)
        exp.init_streams()
        exp.reset()
        exp.run_loop()
        time.sleep(1) # Wait for filters
        data = data_at_volt(wr.filename, volt)
        finish = data.size >= max_points*2
        forward = stop_measure(data, start_state=start_state, threshold=threshold) or finish
    t2 = time.time()
    print("Done one series. Elapsed time: {} min".format((t2-t1)/60))

exp.shutdown_instruments()
print("Done measurement.")

Now at: 0.5.
Measurement count = 1
Done one series. Elapsed time: 0.1991751233736674 min
Now at: 0.45.
Measurement count = 1
Done one series. Elapsed time: 0.19839480717976887 min
Now at: 0.4.
Measurement count = 1
Measurement count = 2
Measurement count = 3
Measurement count = 4
Measurement count = 5
Measurement count = 6
Measurement count = 7
Measurement count = 8
Measurement count = 9
Measurement count = 10
Measurement count = 11
Measurement count = 12
Measurement count = 13
Measurement count = 14
Measurement count = 15
Measurement count = 16
Done one series. Elapsed time: 3.3470255732536316 min
Reached maximum points. Exit.

Done measurement.


In [8]:
# Plot data
volts, data = load_SWR_data(wr.filename)
results = [sw.switching_BER(datum, start_state=start_state, threshold=threshold) for datum in data]
fig = plot_SWR(volts, results)
plt.show()